In [82]:
import sys
import argparse
import configparser
from pathlib import Path

import logging

# see this for inspiration https://github.com/szymonlipinski/examples/blob/master/python_settings/fixed.py

In [3]:
logging.basicConfig(level=logging.DEBUG, format='%(name)s:%(funcName)s %(levelname)s: %(message)s')
logger = logging.getLogger(__name__)


In [351]:
class file():
    def __init__(self, file=None):
        self.file = file
        
    @property
    def file(self):
        return self._file
    
    @file.setter
    def file(self, file):
        if file:
            self._file = Path(file).expanduser().resolve()
        else:
            self._file = None
    # override __repr__ and always output the stored file when called
    def __repr__(self):
        return(repr(self.file))

In [352]:
class Options():
    '''parse command line options
    Args:
        args(`list`): sys.argv is typically passed here
    Properties:
        parser(`argparse.ArgumentParser`): argument parser object
        args(`list`): list of arguments
        options(NameSpace): argument parser generated namespace of arguments
        opts_dict(`dict`): namespace -> dictionary'''
    def __init__(self, args):
        self.parser = argparse.ArgumentParser()      
        self.args = args
    
    @property
    def parser(self):
        '''The argparser object'''
        return self._parser
    
    @parser.setter
    def parser(self, parser):
        if parser:
            self._parser = parser
    
    @property
    def options(self):
        '''argparser namespace of the parsed arguments'''
        try:
            return self._options
        except AttributeError as e:
            self._parse_args()
            return self._options

    
    @options.setter
    def options(self, options):
        if options:
            self._options = options
#             self.opts_dict = vars(self.options)
        else:
            self._options = None
    
    @property
    def opts_dict(self):
        '''namespace of dictionary of parsed options'''
        self._parse_args()
        self._opts_dict = vars(self.options)
        return self._opts_dict
    
    def _parse_args(self):
        '''parse known arguments and discard unknown arguments'''
        options, unknown = self.parser.parse_known_args()
        logging.info(f'discarding unknwon commandline arguments: {unknown}')
        self.options = options
    
    def add_argument(self, *args, **kwargs):
        '''add arguments to the parser using standard argparse.ArgumentParser
        Args:
            *args, **kwargs'''
        try:
            self.parser.add_argument(*args, **kwargs)
        except argparse.ArgumentError as e:
            logging.warning(f'failed adding conflicting option {e}')

In [353]:
class ConfigFile():
    def __init__(self, cfg_file=None):
        self.cfg_file = cfg_file
    
    @property
    def cfg_file(self):
        '''file() class object for storing resolved file path

        Args:
            cfg_file(`str` or `pathlib.Path`): path to .INI type configuration file

        Sets:
            config(`configparser.ConfigParser`): configuration parser object
            config_dict(`dict`): dictionary rpresentation of configuration file'''        
        return self._cfg_file
    
    @cfg_file.setter
    def cfg_file(self, cfg_file):
        self._cfg_file = file(cfg_file)
        self.config = self.cfg_file.file

    @property
    def config(self):
        '''configparser.ConfigParser object for managing configuration file
        Args:
            cfg_file(`str` or `pathlib.Path`): configuration file to parse
        Sets:
            config(`configparser.ConfigParser`): configuration parser object
            config_dict(`dict`): dictionary rpresentation of configuration file'''            
        return self._config
    
    @config.setter
    def config(self, cfg_file):
        config = configparser.ConfigParser()
        config.read(cfg_file)
        self._config = config
        self.config_dict = self._config_2dict(config)
    
    
        
    def _config_2dict(self, configuration):
        '''convert an argparse object into a dictionary

        Args:
            configuration(`configparser.ConfigParser`)

        Returns:
            `dict`'''
        d = {}
        for section in configuration.sections():
            d[section] = {}
            for opt in configuration.options(section):
                d[section][opt] = configuration.get(section, opt)

        return d

In [354]:
c = ConfigFile(cfg_file='./slimpi.cfg')

In [355]:
c.config.sections()

['main', 'lms_server', 'layouts', 'logging']

In [356]:
c.config_dict

{'main': {'splash_screen': 'True'},
 'lms_server': {'host': '', 'port': '9000', 'player_id': ''},
 'layouts': {'display': 'epd5in83',
  'now_playing': 'threeRow',
  'stopped': 'clock',
  'splash': 'splash'},
 'logging': {'log_level': 'INFO'}}

In [357]:
o = Options(sys.argv)

In [358]:
o.add_argument('-p', action='store_true', default=True, help='stuff and things')

In [359]:
o.options

root:_parse_args INFO: discarding unknwon commandline arguments: ['-f', '/home/pi/.local/share/jupyter/runtime/kernel-58d5447b-779d-4244-94e3-e336c9cf9567.json']


Namespace(p=True)

In [81]:
o.opts_dict

root:_parse_args INFO: discarding unknwon commandline arguments: ['-f', '/home/pi/.local/share/jupyter/runtime/kernel-58d5447b-779d-4244-94e3-e336c9cf9567.json']


{'p': True}

create a "Default" configuration object from the builtin config
Create a "user" configuration based on the user config
merge the default and the user overriding the default with the user version
merge the command line over the top of everything

finally creating a dictionary of options